In [1]:
from sentence_transformers import SentenceTransformer, util
from nltk.tokenize import sent_tokenize
import numpy as np
import pandas as pd

/Users/apple/.pyenv/versions/3.11.6/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
model = SentenceTransformer('bert-base-nli-mean-tokens')

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.99k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/Users/apple/.pyenv/versions/3.11.6/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
# read from Top100_06_14_Cleaned.csv

df = pd.read_csv('Top100_06_14_Cleaned.csv')
policy_text = df['cleaned_text'][0]
policy_sentences = sent_tokenize(policy_text)

In [24]:
# law clause
law_clause = "the categories of personal data shared with third parties"

In [25]:
# embedding
law_embedding = model.encode([law_clause])[0]
policy_embeddings = model.encode(policy_sentences)

# calculate cosine similarity
similarities = util.pytorch_cos_sim(law_embedding, policy_embeddings)[0]

In [26]:
# create a dataframe, one column for sentence, one column for similarity
sent_similarity = pd.DataFrame({'sentence': policy_sentences, 'similarity': similarities})

# sort by similarity
sent_similarity = sent_similarity.sort_values(by='similarity', ascending=False)

# print top 5
sent_similarity.head(5)

,sentence,similarity
31,"Information from third-party sources We may combine the personal information that we receive from you with personal information that we obtain from other third-party sources, such as: Data providers.",0.848016
104,We may share the personal information required for the services you request with third parties designated by you.,0.771613
103,We may share your personal data with third parties where you have instructed us or provided your consent to do so.,0.737758
137,Links to Third-Party Platforms.,0.728527
59,"As part of these activities, we may create aggregated or otherwise deidentified data based on the personal information we collect.",0.721621


In [28]:
# print sent_similarity where 'sentence' contains 'third partie'
sent_similarity[sent_similarity['sentence'].str.contains('third partie')]

,sentence,similarity
104,We may share the personal information required for the services you request with third parties designated by you.,0.771613
103,We may share your personal data with third parties where you have instructed us or provided your consent to do so.,0.737758
100,Certain of these disclosures to third parties are known as sharing or processing your personal information for targeted ads and but sharing may be considered selling your personal information under applicable law.,0.677202
138,"The Service may contain links to websites, mobile applications, and other online services operated by third parties.",0.606624
49,Some of these tools may enable us or third parties to collect information about how you act with our and other websites over time.,0.539177
142,"Moreover, we do not control websites, mobile applications or online services operated by third parties, and we are not responsible for their actions.",0.255121
